In [1]:
from laspy.file import File
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the data using laspy
inFile_cloud = File('../../Data/10552_NYU_M2 - Scanner 1 - 190511_164239_1 - originalpoints.laz', mode='r')
inFile_fwf = File('../../Data/164239/10552_NYU_M2 - Scanner 1 - 190511_164239_1 - originalpoints.las', mode='r')
I = inFile_cloud.Classification == 2

# raw is a 1-D numpy array, each entry is a point
raw_cloud = inFile_cloud.get_points()
raw_fwf = inFile_fwf.get_points()
print("Number of points, point cloud: ",len(raw_cloud))
print("Number of points, full waveform: ",len(raw_fwf))

Number of points, point cloud:  14542329
Number of points, full waveform:  14542329


In [3]:
# Corresponds to LAS 1.4 Point Data Record Format 9
# X coordinate of the point (must be adjusted by offset and scale factor)
columns_fwf =[ 
    'X', 
    'Y',
    'Z',
    'intensity', #Intensity of the return, scaled to a 0-65,535 scale
    'flag_byte', #Return number, # of returns, Scan direction flag (+ or -), Edge of flight line flag
    'classification_flags', #Only if the point has been classified: vegetation, etc.
    'classification_byte', #Various flags about the point
    'user_data', 
    'scan_angle', # Down is 0.0, each unit is 0.006 degrees, 
    #so value between -30,000 and 30,000, or (180deg/0.006deg)
    #Spec says this incorporates the roll of the aircraft
    'pt_src_id', #Unique ID for the source of the point
    'gps_time', #GPS Week Time (seconds, reset each week) or Adj Std GPS Time
    'wave_packet_desc_index', # Indicates which waveform packet descriptor is applied.
    'byte_offset_to_waveform_data', # Locates the WF data in EVLR or .wdp file.
    'wavefm_pkt_size', # Size of the waveform packet
    'return_point_waveform_loc', # Time in picoseconds 
    'x_t', # Form parametric location: X = X_0 + x_t*t
    'y_t', # (X_0,Y_0,Z_0) is the anchor point associated with the waveform
    'z_t' ]

# Corresponds to LAS 1.4 Point Data Record Format 6
columns_point_cloud = [
    'X',
    'Y',
    'Z',
    'intensity',
    'flag_byte',
    'classification_flags',
    'classification_byte',
    'user_data',
    'scan_angle',
    'pt_src_id',
    'gps_time']

In [4]:
def raw_to_df(raw,column_names):
    # function takes raw output of laspy.File.get_points() and column names, and returns a pandas Dataframe
    raw_list = [a[0].tolist() for a in raw]
    df = pd.DataFrame(raw_list,columns = column_names)
    return df

def scale_and_offset(df,header):
    '''Function takes as input the dataframe output of raw_to_df and the laspy header file.
       Output is a nx3 dataframe with adjusted X,Y, and Z coordinates, from the formula: 
       X_adj = X*X_scale + X_offset.
       Brooklyn LiDAR readings appear to be in feet, and use NAVD 88 in the vertical and 
       New York Long Island State Plane Coordinate System NAD 33 in the horizontal.'''
    offset = header.offset
    scale = header.scale
    scaled_xyz = df[['X','Y','Z']]*scale + offset
    return scaled_xyz

#INACTIVE FUNCTIONS
def split_and_strip(raw):
    '''Function takes the output ndarray of laspy File.get_points(), 
        splits each point on commas, 
        outputs a list of lists.'''
    output_list = []
    for row in raw:
        stripped = [a.strip("( , )") for a in str(row).split(',')]
        output_list.append(stripped)
    return output_list

def row_clean(string_list):
    '''Function takes as input a list of lists, the output of split_and_strip().  Cleans the entries for spaces,
    returns a numpy array where each row is a point.'''
    output_list = []
    for row in string_list:        
        output_list.append(list(map(float,row[:-1])))
    output_arr = np.array(output_list)
    return output_arr


In [5]:
# Create dataframes for point cloud and fwf
df_cloud = raw_to_df(raw_cloud,columns_point_cloud)
df_fwf = raw_to_df(raw_fwf,columns_fwf)

# Generate adjusted coordinates for point cloud.  FWF is identical.
scaled_xyz = scale_and_offset(df_cloud,inFile_cloud.header)

In [8]:
scaled_xyz_fwf.describe()

,X,Y,Z
count,1.454233e+07,1.454233e+07,1.454233e+07
mean,9.771940e+05,1.730577e+05,5.999902e+01
std,3.226863e+02,1.420471e+03,4.352957e+01
min,9.762879e+05,1.706690e+05,-5.818360e+02
25%,9.769411e+05,1.718709e+05,2.153000e+01
50%,9.772061e+05,1.728613e+05,5.671300e+01
75%,9.774655e+05,1.741343e+05,9.140700e+01
max,9.779850e+05,1.762783e+05,6.671120e+02


In [7]:
# Digging into the VLRs
vlr = inFile_cloud.header.vlrs[0]
vlr.__dir__()
vlr.parsed_body

(1,
 1,
 0,
 4,
 1024,
 0,
 1,
 1,
 1025,
 0,
 1,
 1,
 3072,
 0,
 1,
 32767,
 3076,
 0,
 1,
 9003,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0)